In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from bert4keras.backend import keras, set_gelu
from bert4keras.tokenizers import Tokenizer
from bert4keras.models import build_transformer_model
from bert4keras.optimizers import Adam, extend_with_piecewise_linear_lr
from bert4keras.snippets import sequence_padding, DataGenerator
from bert4keras.snippets import open
from keras.layers import Lambda, Dense

Using TensorFlow backend.


In [2]:
def load_data(filename1,filename2):
    """加载数据
    单条格式：(文本, 标签id)
    """
    data1 = pd.read_excel(filename1)
    data2 = pd.read_excel(filename2)
    
    label_set = list(set(list(data1['label'])+list(data2['label'])))
    label_set.sort()
    print("num_classes:",len(label_set),label_set)
    id_label = {}
    for i in range(len(label_set)):
        id_label[label_set[i]] = i
    data1['label_id'] = data1['label'].map(id_label)
    data2['label_id'] = data2['label'].map(id_label)
    
    D1 = []
    for index,row in data1.iterrows():
        D1.append((row['session_content'], int(row['label_id'])))
    D2 = []
    for index,row in data2.iterrows():
        D2.append((row['session_content'], int(row['label_id'])))
    return D1,D2,label_set

# 加载数据集
train_data,valid_data,label_list = load_data('traindata.xlsx','validdata.xlsx')

num_classes: 10 ['1@1', '1@307', '1@516', '2@301', '2@302', '2@304', '2@305', '2@306', '2@308', '2@309']


In [4]:
set_gelu('tanh')  # 切换gelu版本
num_classes = len(label_list)
maxlen = 512
batch_size = 32

# 下载预训练权重：https://github.com/brightmart/albert_zh
config_path = 'albert_small_zh/albert_config_small_google.json'
checkpoint_path = 'albert_small_zh/albert_model.ckpt'
dict_path = 'albert_small_zh/vocab.txt'

class data_generator(DataGenerator):
    """数据生成器
    """
    def __iter__(self, random=False):
        batch_token_ids, batch_segment_ids, batch_labels = [], [], []
        for is_end, (text, label) in self.sample(random):
            token_ids, segment_ids = tokenizer.encode(text, maxlen=maxlen)
            batch_token_ids.append(token_ids)
            batch_segment_ids.append(segment_ids)
            batch_labels.append([label])
            if len(batch_token_ids) == self.batch_size or is_end:
                batch_token_ids = sequence_padding(batch_token_ids)
                batch_segment_ids = sequence_padding(batch_segment_ids)
                batch_labels = sequence_padding(batch_labels)
                yield [batch_token_ids, batch_segment_ids], batch_labels
                batch_token_ids, batch_segment_ids, batch_labels = [], [], []

# 转换数据集
train_generator = data_generator(train_data, batch_size)
valid_generator = data_generator(valid_data, batch_size)

# 建立分词器
tokenizer = Tokenizer(dict_path, do_lower_case=True)

class data_generator(DataGenerator):
    """数据生成器
    """
    def __iter__(self, random=False):
        batch_token_ids, batch_segment_ids, batch_labels = [], [], []
        for is_end, (text, label) in self.sample(random):
            token_ids, segment_ids = tokenizer.encode(text, maxlen=maxlen)
            batch_token_ids.append(token_ids)
            batch_segment_ids.append(segment_ids)
            batch_labels.append([label])
            if len(batch_token_ids) == self.batch_size or is_end:
                batch_token_ids = sequence_padding(batch_token_ids)
                batch_segment_ids = sequence_padding(batch_segment_ids)
                batch_labels = sequence_padding(batch_labels)
                yield [batch_token_ids, batch_segment_ids], batch_labels
                batch_token_ids, batch_segment_ids, batch_labels = [], [], []


# 加载预训练模型
bert = build_transformer_model(
    config_path=config_path,
    checkpoint_path=checkpoint_path,
    model='albert',
    return_keras_model=False,
)

output = Lambda(lambda x: x[:, 0], name='CLS-token')(bert.model.output)
output = Dense(
    units=num_classes,
    activation='softmax',
    kernel_initializer=bert.initializer
)(output)

model = keras.models.Model(bert.model.input, output)
model.summary()

# 派生为带分段线性学习率的优化器。
# 其中name参数可选，但最好填入，以区分不同的派生优化器。
AdamLR = extend_with_piecewise_linear_lr(Adam, name='AdamLR')

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(1e-5),  # 用足够小的学习率
#     optimizer=AdamLR(learning_rate=1e-4, lr_schedule={
#         1000: 1,
#         2000: 0.1
#     }),
    metrics=['accuracy'],
)

def evaluate(data):
    total, right = 0., 0.
    for x_true, y_true in data:
        y_pred = model.predict(x_true).argmax(axis=1)
        y_true = y_true[:, 0]
        total += len(y_true)
        right += (y_true == y_pred).sum()
    return right / total

class Evaluator(keras.callbacks.Callback):
    """评估与保存
    """
    def __init__(self):
        self.best_val_acc = 0.

    def on_epoch_end(self, epoch, logs=None):
        val_acc = evaluate(valid_generator)
        if val_acc > self.best_val_acc:
            self.best_val_acc = val_acc
            model.save_weights('best_model.weights')
#         test_acc = evaluate(test_generator)
        print(
            u'val_acc: %.5f, best_val_acc: %.5f\n' %
            (val_acc, self.best_val_acc)
        )

if False:
    evaluator = Evaluator()
    model.fit(
        train_generator.forfit(),
        steps_per_epoch=len(train_generator),
        epochs=20,
        callbacks=[evaluator]
    )
    model.load_weights('best_model.weights')

else:

    model.load_weights('best_model.weights')
#     val_acc = evaluate(valid_generator)
#     print(val_acc)

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, None)         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     (None, None, 128)    2704384     Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, None, 128)    256         Input-Segment[0][0]              
____________________________________________________________________________________________

In [4]:
!rm pb/ner_model.pb
!rm -rf saved_model1/*

In [5]:
from keras import backend as K
import tensorflow as tf
# convert .h5 to .pb
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,output_names, freeze_var_names)
        return frozen_graph

print('input is :', (model.input[0].name,model.input[1].name))
print ('output is:', model.output.name)
sess = K.get_session()
frozen_graph = freeze_session(K.get_session(), output_names=[model.output.op.name])

input is : ('Input-Token:0', 'Input-Segment:0')
output is: dense_7/Softmax:0
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 30 variables.
INFO:tensorflow:Converted 30 variables to const ops.


In [6]:
from tensorflow.python.framework import graph_io

output_path='./pb'
pb_model_name='ner_model.pb'
# pb_model_name='t5/t5_model.pb'
graph_io.write_graph(frozen_graph, output_path, pb_model_name, as_text=False)

'./pb/ner_model.pb'

In [7]:
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import tag_constants
# export_dir = 't5/saved_model'
# # graph_pb = './5_trained_model.pb'
# graph_pb = 't5/t5_model.pb'
export_dir = './saved_model1'
# graph_pb = './5_trained_model.pb'
graph_pb = './pb/ner_model.pb'
builder = tf.saved_model.builder.SavedModelBuilder(export_dir)
with tf.gfile.GFile(graph_pb, "rb") as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

In [8]:
sigs = {}
with tf.Session(graph=tf.Graph()) as sess:
    tf.import_graph_def(graph_def, name="")
    g = tf.get_default_graph()
    inp0 = g.get_tensor_by_name(model.input[0].name)
    inp1 = g.get_tensor_by_name(model.input[1].name)
    out = g.get_tensor_by_name(model.output.name)
    print('inp0',inp0)
    print('inp1',inp1)
    print('out',out)
    
    sigs[signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY] = tf.saved_model.signature_def_utils.predict_signature_def({"in0":inp0,"in1":inp1},{"out":out})
    builder.add_meta_graph_and_variables(sess,[tag_constants.SERVING],signature_def_map=sigs)
builder.save()

inp0 Tensor("Input-Token:0", shape=(?, ?), dtype=float32)
inp1 Tensor("Input-Segment:0", shape=(?, ?), dtype=float32)
out Tensor("dense_7/Softmax:0", shape=(?, 10), dtype=float32)
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./saved_model1/saved_model.pb


b'./saved_model1/saved_model.pb'